In [1]:
using Random, Statistics, PyPlot, Polynomials

copying from PS2

In [2]:
struct Point2D
    x::Int64
    y::Int64
end

struct SquareLattice
    Nx::Int64
    Ny::Int64    
end

import Base.length
function length(lat::SquareLattice)
    return lat.Nx*lat.Ny
end

function PointToIndex(lat::SquareLattice, r::Point2D)
    @assert r.x <= lat.Nx "Site x coordinate exceeding lattice length"
    @assert r.y <= lat.Ny "Site y coordinate exceeding lattice width"
    
    return (r.y-1)*lat.Nx + r.x
end

function IndexToPoint(lat::SquareLattice, ind::Int64)
    
    @assert ind <= length(lat) "Site index exceeding lattice size"
    
    y = div(ind-1, lat.Nx)+1
    x = mod(ind-1, lat.Nx)+1

    return Point2D(x,y)
end
    
function NearestNeighbors(lat::SquareLattice, ind::Int64)

    r = IndexToPoint(lat, ind)
    
    nn = zeros(Int64,4)
    
    xp = mod(r.x, lat.Nx)+1
    yp = mod(r.y, lat.Ny)+1
    xm = mod(r.x-2, lat.Nx)+1
    ym = mod(r.y-2, lat.Ny)+1
    
    nn[1] = PointToIndex(lat, Point2D(xm, r.y) )
    nn[2] = PointToIndex(lat, Point2D(xp, r.y) )
    nn[3] = PointToIndex(lat, Point2D(r.x, ym) )
    nn[4] = PointToIndex(lat, Point2D(r.x, yp) )
    
    return nn
end

function NearestNeighbors(lat::SquareLattice, ind::Int64,direction::Int64)

    r = IndexToPoint(lat, ind)
    
    nn = zeros(Int64,2)
    
    if direction == 0    
        xp = mod(r.x, lat.Nx)+1
        xm = mod(r.x-2, lat.Nx)+1
        nn[1] = PointToIndex(lat, Point2D(xm, r.y) )
        nn[2] = PointToIndex(lat, Point2D(xp, r.y) )
    end
    
    if direction == 1 
        yp = mod(r.y, lat.Ny)+1
        ym = mod(r.y-2, lat.Ny)+1
        nn[1] =  PointToIndex(lat, Point2D(r.x, ym) )
        nn[2] = PointToIndex(lat, Point2D(r.x, yp) )
    end
    
    return nn
end



NearestNeighbors (generic function with 2 methods)

In [8]:
function ConfigurationEnergy(lat::SquareLattice, c::Vector{Float64},Jx::Float64,Jtau::Float64)
    
    N = length(lat)
    @assert N == length(c) "Configuration incompatible with lattice size"
    
    Ec = 0.
    throw("Not Implemented")
    
    
    for j in range(1, length = N)
        Xnn = NearestNeighbors(lat, j, 0)
        taunn = NearestNeighbors(lat, j, 1)
        #Ec += -sum(map(x->cos(c[j]-c[x]),nn))
        Ec += Jx * sum(c[j] .*c[Xnn])
        Ec += Jtau * sum(c[j] .*c[taunn])
        
    end
    return Ec/2 # in the summation above every bond is included 2 times
    
end

ConfigurationEnergy (generic function with 1 method)

In [8]:
function WolffUpdate!(lat::SquareLattice, c::Vector{Float64}, T::Float64)

    N = length(lat)
    
    @assert N == length(c) "Configuration incompatible with lattice size"
    
    
    
    #define a random direction: ̂e
    direction = rand(Float64)*2π
    
    
    

    cluster = Int64[] # indices of spins in the cluster
    newlyAdded = Int64[] # indices of newly added neighbors

    push!(cluster, ind)
    append!(newlyAdded, NearestNeighbors(lat, ind))
    
    
    # differently for different directions?
    throw("Not Implemented")
    while !isempty(newlyAdded)
        j = pop!(newlyAdded)
        if j in cluster
            continue
        end
        
        p = 1. - exp(-2/T * c[ind]*c[j])
        if rand() < p # if si_e and sj_e are unti-aligned  p will be negative (i might want to check this!🎈)    
            push!(cluster, j)
            nn = NearestNeighbors(lat, j)
            append!(newlyAdded, nn)
        end
    end

    #flip the cluster!
    ## theta'  = -theta -2alpha + pi
    c[cluster] = map(x->mod(x,2pi),-c[cluster] .- 2 * direction .+ pi)
    #c[cluster] = -c[cluster] .- 2 * direction .+ pi
end

WolffUpdate! (generic function with 1 method)

In [7]:
function QMC(lat::SquareLattice, T::Float64, Nsw::Int64, saveConfigs = false)
    # Nsw is the number of sweeps to perform
    
    c = rand(0:1, length(lat))

    if saveConfigs
        configs = zeros(Float64, Nsw, length(lat))
        configs[1, :] = c
    end
        
    
    m[1] = abs( mean(2*c .- 1) )
    throw("here we need to avg on the absolut magnetization of each row!!")
    
    throw("Not implemented")
    
    En = zeros(Float64, Nsw)
    En[1] = ConfigurationEnergy(lat, c)
    
    for i in range(2, stop=Nsw)
        WolffUpdate!(lat, c, T)
        
        En[i] = ConfigurationEnergy(lat, c)
        
        if saveConfigs
            configs[i, :] = c
        end
        
        
        throw("here we need to avg on the absolut magnetization of each row!!")
        m[i] = abs( mean(2*c .- 1) )
        
        

    end
    
    if saveConfigs
        return mx, my, En, configs
    else
        return mx, my, En
    end
    
end

QMC (generic function with 2 methods)